<div class="alert alert-success">
<h1>Local encoding of prediction error signals</h1>
<hr>
<p>
I(HGA; PE)
</p>
</div>

In [ ]:

import numpy as np
import xarray as xr
import pandas as pd

from pathta import Study

from frites import set_mpl_style
from frites.utils import kernel_smoothing

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D

from ipywidgets import interact
import matplotlib.image as mpimg

import json
with open("config.json", 'r') as f:
    config = json.load(f)

set_mpl_style()
plt.rcParams['xtick.labelsize'] = 'xx-large'
plt.rcParams['ytick.labelsize'] = 'xx-large'
plt.rcParams['axes.titlesize'] = 23
plt.rcParams['axes.labelsize'] = 22


---
# **I/O**

In [ ]:
###############################################################################
model = 'PE'
band = 'lga'
from_folder = f'mi/group/{model}'
###############################################################################

st = Study('PBLT')

# load significance testing
f = st.search('savgol-10.nc', band, folder=from_folder)
assert len(f) == 1
dt = xr.load_dataset(f[0])

# load conjunction
f = st.search('conj', band, folder=from_folder)
assert len(f) == 1
conj = xr.load_dataset(f[0]).to_array('cond').sel(cond=['pun', 'rew'])

model = model.replace('_', ' | ')


---
# **Plotting**
## Group-level MI and non-specific proportion of contacts

**Non-specific** because I'm not looking at contacts significant **only** for rew or **only** for pun 

In [ ]:
# ###############################################################################
# sigma = 0.001
# ci_l = 'sem'
# nb_kw = dict(size=27, weight='bold')
# ###############################################################################


# def prop(x):
#     n_roi = len(x['roi'])
#     return 100 * x.sum('roi') / n_roi


# def shared_prop(x):
#     n_roi = len(x['roi'])
#     is_rew = (x.sel(cond='rew') < 0.05).data
#     is_pun = (x.sel(cond='pun') < 0.05).data
#     is_rp = 100 * np.stack((is_rew, is_pun)).all(0).sum(1) / n_roi

#     return xr.DataArray(is_rp, dims=('times',), coords=(x['times'].data,))

# sstr = "%.3f" % sigma
# sstr = sstr[2:]

# # get variables
# times = dt['times'].data
# conj_prop = fg = (conj < 0.05).groupby('roi').apply(prop)
# n_contacts = conj.isel(cond=0, times=0).groupby('roi').count()

# # compute shared proportion
# is_roi = (conj < 0.05).any('times').any('cond').data
# # conj_shared = conj.sel(roi=is_roi).groupby('roi').apply(shared_prop)  # shared among significant sites
# conj_shared = conj.groupby('roi').apply(shared_prop)  # shared across all significant sites

# # get mi, pv, ci
# mi = xr.Dataset({'rew': dt['mi_rew'], 'pun': dt['mi_pun']}).to_array('cond')
# pv = xr.Dataset({'rew': dt[f'pv_rew_{sstr}'],
#                  'pun': dt[f'pv_pun_{sstr}']}).to_array('cond')
# ci = xr.Dataset({'rew': dt['ci_rew'], 'pun': dt['ci_pun']}).to_array(
#     'cond').sel(ci=ci_l)
# # mi = ci.mean('bound')

# cl = xr.full_like(mi, np.nan)
# minmin = ci.data.min()
# delta = (ci.data.max() - minmin) / 20.
# cl.data[0, pv.data[0, ...] < 0.05] = minmin - delta
# cl.data[1, pv.data[1, ...] < 0.05] = minmin - 2 * delta

# fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(
#     20, 8), sharex=True, sharey='row', gridspec_kw=dict(hspace=.1, wspace=.1))

# for n_r, r in enumerate(['aINS', 'dlPFC', 'lOFC', 'vmPFC']):
#     for n_c, c in enumerate(['rew', 'pun']):
#         color = 'C1' if c == 'rew' else 'C0'
#         # group-level plot
#         plt.sca(axs[0, n_r])
#         plt.plot(times, mi.sel(cond=c, roi=r).data, color=color)
#         plt.fill_between(
#             times, ci.sel(bound='low', cond=c, roi=r).data,
#             ci.sel(bound='high', cond=c, roi=r).data, color=color, alpha=.2,
#             zorder=-1
#         )
#         ln, = plt.plot(times, cl.sel(cond=c, roi=r).data, color=color, lw=6)
#         ln.set_solid_capstyle('round')
#         plt.axvline(0., color='C3')
#         plt.title(f"{r} (n={n_contacts.sel(roi=r).data})", fontweight='bold')
#         if n_r == 0:
#             plt.ylabel('MI (bits)')

#         # proportion plot
#         plt.sca(axs[1, n_r])
#         plt.plot(times, conj_prop.sel(cond=c, roi=r).data, color=color)
#         plt.axvline(0., color='C3')
#         if n_r == 0:
#             plt.ylabel('% contacts')
#         plt.xlabel('Times (s)')
#     plt.plot(times, conj_shared.sel(roi=r).data, color='C2')
# # plt.xlim(-.5, 1.5)
# plt.yticks([0, 5, 10, 15, 20, 25, 30, 35])

# # annotations
# ax = axs[0, 0]
# ax.text(*tuple([-.3, 1.05]), 'A', transform=ax.transAxes, **nb_kw)
# ax = axs[1, 0]
# ax.text(*tuple([-.3, 1.05]), 'B', transform=ax.transAxes, **nb_kw)

# # create the legend
# custom_lines = [
#     Line2D([0], [0], color="C1", lw=6, solid_capstyle='round'),
#     Line2D([0], [0], color="C0", lw=6, solid_capstyle='round'),
#     Line2D([0], [0], color="C2", lw=6, solid_capstyle='round'),
# ]
# titles = [r"$MI_{RPE}$", r"$MI_{PPE}$", r"$Shared_{R/PPE}$"]
# plt.legend(
#     custom_lines, titles, ncol=3, bbox_to_anchor=(.7, 0.02), fontsize=20,
#     bbox_transform=fig.transFigure, title="Significant cluster of MI (p<0.05)",
#     title_fontproperties=dict(weight='bold', size=20)
# );


### Export the figure

In [ ]:
# save_to = config['export']['save_to']
# cfg_export = config['export']['cfg']

# fig.savefig(f'{save_to}/fig_local_mi.png', **cfg_export)

## Group-level MI and specific proportion of contacts

In [ ]:
# ###############################################################################
# sigma = 0.001
# ci_l = 'sem'
# nb_kw = dict(size=27, weight='bold')
# ###############################################################################


# def compute_prop(x):
#     n_roi = len(x['roi'])
#     is_rew, is_pun = x.sel(cond='rew') < 0.05, x.sel(cond='pun') < 0.05
#     is_rew_only = 100 * xr.concat(
#         (is_rew, ~is_pun), 'cond').all('cond').sum('roi') / n_roi
#     is_pun_only = 100 * xr.concat(
#         (~is_rew, is_pun), 'cond').all('cond').sum('roi') / n_roi
#     is_rp = 100 * xr.concat(
#         (is_rew, is_pun), 'cond').all('cond').sum('roi') / n_roi
#     is_none = 100 * xr.concat(
#         (~is_rew, ~is_pun), 'cond').all('cond').sum('roi') / n_roi
    
#     return xr.Dataset({
#         'rew': is_rew_only, 'pun': is_pun_only, 'shared': is_rp,
#         'none': is_none
#     }).to_array('cond')

 
# sstr = "%.3f" % sigma
# sstr = sstr[2:]

# # get variables
# times = dt['times'].data
# conj_prop = conj.groupby('roi').apply(compute_prop)
# n_contacts = conj.isel(cond=0, times=0).groupby('roi').count()

# # get mi, pv, ci
# mi = xr.Dataset({'rew': dt['mi_rew'], 'pun': dt['mi_pun']}).to_array('cond')
# # tv = xr.Dataset({'rew': dt[f'tv_rew_{sstr}'],
# #                  'pun': dt[f'tv_pun_{sstr}']}).to_array('cond')
# pv = xr.Dataset({'rew': dt[f'pv_rew_{sstr}'],
#                  'pun': dt[f'pv_pun_{sstr}']}).to_array('cond')
# ci = xr.Dataset({'rew': dt['ci_rew'], 'pun': dt['ci_pun']}).to_array(
#     'cond').sel(ci=ci_l)
# # mi = ci.mean('bound')

# cl = xr.full_like(mi, np.nan)
# minmin = ci.data.min()
# delta = (ci.data.max() - minmin) / 20.
# cl.data[0, pv.data[0, ...] < 0.05] = minmin - delta
# cl.data[1, pv.data[1, ...] < 0.05] = minmin - 2 * delta

# fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(
#     20, 8), sharex=False, sharey='row', gridspec_kw=dict(hspace=.2, wspace=.1))

# for n_r, r in enumerate(['aINS', 'dlPFC', 'lOFC', 'vmPFC']):
#     for n_c, c in enumerate(['rew', 'pun']):
#         color = 'C1' if c == 'rew' else 'C0'
#         # group-level plot
#         plt.sca(axs[0, n_r])
#         plt.plot(times, mi.sel(cond=c, roi=r).data, color=color)
#         plt.fill_between(
#             times, ci.sel(bound='low', cond=c, roi=r).data,
#             ci.sel(bound='high', cond=c, roi=r).data, color=color, alpha=.2,
#             zorder=-1
#         )
#         ln, = plt.plot(times, cl.sel(cond=c, roi=r).data, color=color, lw=6)
#         ln.set_solid_capstyle('round')
#         plt.axvline(0., color='C3')
#         plt.title(f"{r} (n={n_contacts.sel(roi=r).data})", fontweight='bold')
#         if n_r == 0:
#             plt.ylabel('MI (bits)')

#         # proportion plot
#         plt.sca(axs[1, n_r])
#         plt.plot(times, conj_prop.sel(cond=c, roi=r).data, color=color)
#         plt.axvline(0., color='C3')
#         if n_r == 0:
#             plt.ylabel('% contacts')
#         plt.xlabel('Times (s)')
#     plt.plot(times, conj_prop.sel(roi=r, cond='shared').data, color='C2')
#     plt.plot(times, conj_prop.sel(roi=r, cond='none').data, color='C3')
# # plt.xlim(-.5, 1.5)
# # plt.yticks([0, 5, 10, 15, 20, 25, 30])
# plt.yticks([0, 10, 20, 30, 50, 75, 100])

# # annotations
# ax = axs[0, 0]
# ax.text(*tuple([-.3, 1.05]), 'A', transform=ax.transAxes, **nb_kw)
# ax = axs[1, 0]
# ax.text(*tuple([-.3, 1.05]), 'B', transform=ax.transAxes, **nb_kw)

# # create the legend
# custom_lines = [
#     Line2D([0], [0], color="C1", lw=6, solid_capstyle='round'),
#     Line2D([0], [0], color="C0", lw=6, solid_capstyle='round'),
#     Line2D([0], [0], color="C2", lw=6, solid_capstyle='round'),
# ]
# titles = [r"$MI_{RPE}$", r"$MI_{PPE}$", r"$Shared_{R/PPE}$"]
# plt.legend(
#     custom_lines, titles, ncol=3, bbox_to_anchor=(.7, 0.02), fontsize=20,
#     bbox_transform=fig.transFigure, title="Significant cluster of MI (p<0.05)",
#     title_fontproperties=dict(weight='bold', size=20)
# );


### Export the figure

In [ ]:
# save_to = config['export']['save_to']
# cfg_export = config['export']['cfg']

# fig.savefig(f'{save_to}/fig_local_mi.png', **cfg_export)


## Group-level MI and specific proportion of contacts (+ custom legend)

In [ ]:
###############################################################################
sigma = 0.001
ci_l = 'sem'
nb_kw = dict(size=27, weight='bold')
###############################################################################


def compute_prop(x):
    n_roi = len(x['roi'])
    is_rew, is_pun = x.sel(cond='rew') < 0.05, x.sel(cond='pun') < 0.05
    is_rew_only = 100 * xr.concat(
        (is_rew, ~is_pun), 'cond').all('cond').sum('roi') / n_roi
    is_pun_only = 100 * xr.concat(
        (~is_rew, is_pun), 'cond').all('cond').sum('roi') / n_roi
    is_rp = 100 * xr.concat(
        (is_rew, is_pun), 'cond').all('cond').sum('roi') / n_roi
    is_none = 100 * xr.concat(
        (~is_rew, ~is_pun), 'cond').all('cond').sum('roi') / n_roi

    return xr.Dataset({
        'rew': is_rew_only, 'pun': is_pun_only, 'shared': is_rp,
        'none': is_none
    }).to_array('cond')


def plt_img(name, ax):
    img_intra = mpimg.imread(f'images/{name}.png')
    plt.imshow(img_intra)
    plt.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])

sstr = "%.3f" % sigma
sstr = sstr[2:]

# get variables
times = dt['times'].data
conj_prop = conj.groupby('roi').apply(compute_prop)
n_contacts = conj.isel(cond=0, times=0).groupby('roi').count()

# get mi, pv, ci
mi = xr.Dataset({'rew': dt['mi_rew'], 'pun': dt['mi_pun']}).to_array('cond')
# tv = xr.Dataset({'rew': dt[f'tv_rew_{sstr}'],
#                  'pun': dt[f'tv_pun_{sstr}']}).to_array('cond')
pv = xr.Dataset({'rew': dt[f'pv_rew_{sstr}'],
                 'pun': dt[f'pv_pun_{sstr}']}).to_array('cond')
ci = xr.Dataset({'rew': dt['ci_rew'], 'pun': dt['ci_pun']}).to_array(
    'cond').sel(ci=ci_l)
# mi = ci.mean('bound')

cl = xr.full_like(mi, np.nan)
minmin = ci.data.min()
delta = (ci.data.max() - minmin) / 20.
cl.data[0, pv.data[0, ...] < 0.05] = minmin - delta
cl.data[1, pv.data[1, ...] < 0.05] = minmin - 2 * delta

fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(
    20, 8), sharex=False, sharey='row', gridspec_kw=dict(hspace=.2, wspace=.1))

for n_r, r in enumerate(['aINS', 'dlPFC', 'lOFC', 'vmPFC']):
    for n_c, c in enumerate(['rew', 'pun']):
        color = 'C1' if c == 'rew' else 'C0'
        # group-level plot
        plt.sca(axs[0, n_r])
        plt.plot(times, mi.sel(cond=c, roi=r).data, color=color)
        plt.fill_between(
            times, ci.sel(bound='low', cond=c, roi=r).data,
            ci.sel(bound='high', cond=c, roi=r).data, color=color, alpha=.2,
            zorder=-1
        )
        ln, = plt.plot(times, cl.sel(cond=c, roi=r).data, color=color, lw=6)
        ln.set_solid_capstyle('round')
        plt.axvline(0., color='C3')
        plt.title(f"{r} (n={n_contacts.sel(roi=r).data})", fontweight='bold')
        if n_r == 0:
            plt.ylabel('MI [bits]')

        # proportion plot
        plt.sca(axs[1, n_r])
        plt.plot(times, conj_prop.sel(cond=c, roi=r).data, color=color)
        plt.axvline(0., color='C3')
        if n_r == 0:
            plt.ylabel('Bipolar\nderivations [%]')
        plt.xlabel('Times [s]')
    plt.plot(times, conj_prop.sel(roi=r, cond='shared').data, color='C2')
    plt.plot(times, conj_prop.sel(roi=r, cond='none').data, color='C3')
# plt.xlim(-.5, 1.5)
# plt.yticks([0, 5, 10, 15, 20, 25, 30])
plt.yticks([0, 10, 20, 30, 50, 75, 100])

# annotations
ax = axs[0, 0]
ax.text(*tuple([-.3, 1.05]), 'A', transform=ax.transAxes, **nb_kw)
ax = axs[1, 0]
ax.text(*tuple([-.3, 1.05]), 'B', transform=ax.transAxes, **nb_kw)

# add proportion legend
ax = fig.add_axes([0.1, -0.24, .8, .25])
plt_img('task-related', ax)

# # create the legend
# custom_lines = [
#     Line2D([0], [0], color="C1", lw=6, solid_capstyle='round'),
#     Line2D([0], [0], color="C0", lw=6, solid_capstyle='round'),
#     Line2D([0], [0], color="C2", lw=6, solid_capstyle='round'),
# ]
# titles = [r"$MI_{RPE}$", r"$MI_{PPE}$", r"$Shared_{R/PPE}$"]
# plt.legend(
#     custom_lines, titles, ncol=3, bbox_to_anchor=(.7, 0.02), fontsize=20,
#     bbox_transform=fig.transFigure, title="Significant cluster of MI (p<0.05)",
#     title_fontproperties=dict(weight='bold', size=20)
# )


In [ ]:
save_to = config['export']['save_to']
cfg_export = config['export']['cfg']

fig.savefig(f'{save_to}/fig_local_mi.png', **cfg_export)


In [ ]:
50 + 50 + 75 + 65